In [ ]:
source_folder = "./source"
video_file = 'nameofvideo1.mp4'
video_name = video_file.split('.')[0]
dataset_out_folder = './extracted'

base_json_file = source_folder + '/' + video_name + '.funscript'
surge_json_file = source_folder + '/' + video_name + '.surge.funscript'
sway_json_file = source_folder + '/' + video_name + '.sway.funscript'
pitch_json_file = source_folder + '/' + video_name + '.pitch.funscript'
roll_json_file = source_folder + '/' + video_name + '.roll.funscript'
twist_json_file = source_folder + '/' + video_name + '.twist.funscript'

timestamp_file = source_folder + '/' + video_name + '.txt'
video_file_path = source_folder + '/' + video_file

import cv2
import numpy as np
import json
import os

base_json = json.load(open(base_json_file))["actions"]
surge_json = json.load(open(surge_json_file))["actions"]
sway_json = json.load(open(sway_json_file))["actions"]
pitch_json = json.load(open(pitch_json_file))["actions"]
roll_json = json.load(open(roll_json_file))["actions"]
spin_json = json.load(open(twist_json_file))["actions"]

positions_data = [base_json, surge_json, sway_json, pitch_json, roll_json, spin_json]


In [ ]:
timestamps = []
with open(timestamp_file, 'r') as f:
    for line in f:
        clean_line = line.strip()
        line_parts = clean_line.split(' ')
        start = (int(line_parts[1].split(':')[0]) * 60 + int(line_parts[1].split(':')[1])) * 1000
        end = (int(line_parts[2].split(':')[0]) * 60 + int(line_parts[2].split(':')[1])) * 1000
        timestamps.append((start, end))
print(timestamps)

In [ ]:
cap = cv2.VideoCapture(video_file_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) // 2
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

In [ ]:
print(video_file)
print(video_file_path)
print(video_name)
print(frame_width, frame_height, fps, frame_count)

In [ ]:
import os
from tqdm import tqdm

def get_data_at_time(time_, positions_):
    pos_data = []
    for axis in positions_:
        # find the "at" values in the json that the time is between
        previous_action = None
        next_action = None
        for i in range(len(axis) -1):
            if axis[i]["at"] <= time_ <= axis[i+1]["at"]:
                previous_action = axis[i]
                next_action = axis[i+1]
        if previous_action is None or next_action is None:
            print("Error: time not found in data")
            return
        previous_pos = previous_action["pos"]
        next_pos = next_action["pos"]
        previous_at = previous_action["at"]
        next_at = next_action["at"]

        time_difference = next_at - previous_at
        pos_difference = next_pos - previous_pos

        current_time_difference = time_ - previous_at
        current_pos_difference = current_time_difference * (pos_difference / time_difference)
        pos_data.append((previous_pos + current_pos_difference) / 100.0)
    return pos_data
k = 0
frame_batch = []
frame_batch_size = 10

active_timestamp = timestamps[0] # forget if I need to fix this to loop them all

cap.set(cv2.CAP_PROP_POS_MSEC, active_timestamp[0])
while True:
    ret, frame = cap.read()
    frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
    frame_timestamp = int(cap.get(cv2.CAP_PROP_POS_MSEC))
    #print(frame_number, frame_timestamp)
    frame_out_file = dataset_out_folder + '/' + video_name + '_frame_' + str(frame_number) + '.jpg'
    data_out_file = dataset_out_folder + '/' + video_name + '_frame_' + str(frame_number) + '.txt'
    if os.path.exists(frame_out_file) and os.path.exists(data_out_file):
        continue
    if os.path.exists(dataset_out_folder2 + '/' + video_name + '_frame_' + str(frame_number) + '.jpg') and os.path.exists(dataset_out_folder2 + '/' + video_name + '_frame_' + str(frame_number) + '.txt'):
        continue

    if frame_timestamp > active_timestamp[1]:
        print("reach end of first timestamp")
        break
    frame = frame[:, frame_width:]
    #frame = frame[frame_height // 2:, frame_width // 4: frame_width // 4 * 3]

    six_axis_frame_position = get_data_at_time(frame_timestamp, positions_data)
    
    # with open(data_out_file, 'w') as f:
    #     f.write(str(six_axis_frame_position).replace('[', '').replace(']', '').replace(',', '').replace(' ', '\n'))           
    # cv2.imwrite(frame_out_file, frame)

    if not os.path.exists(frame_out_file) or not os.path.exists(data_out_file): 
        frame_batch.append((frame_out_file, frame, data_out_file, six_axis_frame_position))



    if len(frame_batch) == frame_batch_size:
        for i in range(frame_batch_size):
            cv2.imwrite(frame_batch[i][0], frame_batch[i][1])
            #print(frame_batch[i][0])
            with open(frame_batch[i][2], 'w') as f:
                f.write(str(frame_batch[i][3]).replace('[', '').replace(']', '').replace(',', '').replace(' ', '\n'))
                
            k = cv2.waitKey(1) & 0xFF
            if k == ord('q'):
                break

        frame_batch = []
    
    # draw a circle to visualize the position of the axises
    # circle_x = int(six_axis_frame_position[0] * frame_width / 2)
    # circle_y = int(six_axis_frame_position[2] * frame_height /2)
    # frame = cv2.circle(frame, (circle_x, circle_y), 10, (0, 0, 255), -1)
    # circle_x = int(six_axis_frame_position[3] * frame_width / 2 + frame_width / 2)
    # circle_y = int(six_axis_frame_position[1] * frame_height  /2)
    # frame = cv2.circle(frame, (circle_x, circle_y), 10, (0, 255, 0), -1)
    # circle_x = int(six_axis_frame_position[4] * frame_width / 2 )
    # circle_y = int(six_axis_frame_position[5] * frame_height  /2 + frame_height / 2)
    # frame = cv2.circle(frame, (circle_x, circle_y), 10, (255, 0, 0), -1)
    # frame = cv2.resize(frame, (frame_width //2, frame_height//2))
    frame = cv2.resize(frame, (512, 512))
    cv2.imshow('frame', frame)
    k = cv2.waitKey(1) & 0xFF
    if k == ord('q'):
        break

cv2.destroyAllWindows()